In [1]:
#step 1 import packages/libraries
import sklearn as sk 
from sklearn import linear_model   #I am performing a linear regression, if it was multi_lin I would use different packages
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib import style
from sklearn import svm
import numpy as np
%matplotlib inline

#step 2 import csv, I use dwelling 8655 which is the most complete dwelling
dir = '//datc//opschaler//combined_gas_smart_weather_dfs//processed//'
dwelling_id = 'P01S01W5855_hour'
df = pd.read_csv(dir+dwelling_id+'.csv', delimiter='\t', parse_dates=['datetime'])
df = df.set_index(['datetime'])
df.head()

,eMeter,eMeterReturn,eMeterLow,eMeterLowReturn,ePower,ePowerReturn,gasMeter,DD,DR,FF,...,RG,SQ,T,T10,TD,U,VV,WW,dwelling,gasPower
datetime,,,,,,,,,,,,,,,,,,,,,
2017-03-07 07:00:00,NaN,NaN,NaN,NaN,NaN,NaN,2073.335,283.46667,0.0,2.076667,...,0.0,0.0,5.516667,4.400000,4.916666,95.166664,10295.000,4.666666,P01S01W5855,NaN
2017-03-07 08:00:00,2458.056,0.0,2370.064,0.0,477.85837,0.0,2073.933,273.10000,0.0,2.106667,...,0.0,0.0,5.933333,5.083334,5.216667,94.500000,10565.000,3.333333,P01S01W5855,0.598145
2017-03-07 09:00:00,2458.288,0.0,2370.064,0.0,232.49167,0.0,2074.427,281.45000,0.0,2.453333,...,0.0,0.0,6.466667,5.966667,5.800000,95.000000,11416.667,2.000000,P01S01W5855,0.493896
2017-03-07 10:00:00,2459.166,0.0,2370.064,0.0,880.23050,0.0,2074.991,282.10000,100.0,2.848333,...,0.0,0.0,7.016666,6.866667,5.916666,92.000000,11550.000,23.333334,P01S01W5855,0.563965
2017-03-07 11:00:00,2459.424,0.0,2370.064,0.0,251.95273,0.0,2075.398,286.85000,2.0,4.028333,...,0.0,0.0,7.566667,7.716667,6.183333,90.500000,13883.333,32.500000,P01S01W5855,0.406982


In [2]:
df.describe()

,eMeter,eMeterReturn,eMeterLow,eMeterLowReturn,ePower,ePowerReturn,gasMeter,DD,DR,FF,...,Q,RG,SQ,T,T10,TD,U,VV,WW,gasPower
count,1931.000000,1931.000000,1931.000000,1931.000000,1931.000000,1931.000000,1932.000000,1934.000000,1934.000000,1934.000000,...,1934.000000,1934.000000,1934.000000,1934.000000,1934.000000,1934.000000,1934.000000,1934.000000,1934.000000,1928.000000
mean,2576.817495,193.489820,2548.485107,103.322437,255.047603,412.448292,2209.962533,192.154961,29.691210,3.947510,...,180.266891,0.052099,2.917410,10.922064,10.292085,6.494607,75.596329,25985.648410,8.762203,0.110303
std,47.603318,167.781277,95.681948,94.411037,284.384288,783.150219,66.339005,92.202189,108.820337,2.128717,...,231.795511,0.297233,3.955380,5.069890,6.281625,3.727372,15.228334,12813.323562,14.911292,0.195111
min,2458.056000,0.000000,2370.064000,0.000000,0.000000,0.000000,2073.335000,4.450000,0.000000,0.026667,...,0.000000,0.000000,0.000000,-1.700000,-5.916667,-3.133333,30.333334,142.333330,1.000000,0.000000
25%,2547.918000,0.000000,2471.592000,0.000000,47.497604,0.000000,2158.039250,103.375002,0.000000,2.296667,...,0.166667,0.000000,0.000000,7.766667,6.550000,3.450000,65.166664,15356.250000,1.833333,0.000000
50%,2588.717000,164.318000,2561.796000,95.818000,142.487470,0.000000,2214.147000,211.841670,0.000000,3.715833,...,47.500000,0.000000,0.000000,10.333333,10.141666,6.600000,77.666664,26418.333000,2.166667,0.000000
75%,2616.439000,304.706000,2633.858000,192.465000,377.546080,445.411965,2278.816250,263.374998,0.000000,5.296250,...,326.541660,0.000000,6.666666,13.450000,13.866667,9.000000,88.191666,35982.499000,7.291667,0.169922
max,2636.481000,509.856000,2697.496000,308.527000,1856.916400,3360.461200,2293.133000,354.316680,600.000000,12.312000,...,875.500000,5.900000,10.000000,30.683332,32.466667,16.166666,99.166664,49900.000000,81.333336,1.120117


In [13]:
df.isnull().sum()

eMeter             1854
eMeterReturn          3
eMeterLow             3
eMeterLowReturn       3
ePower                3
ePowerReturn          3
gasMeter           1854
DD                    0
DR                    0
FF                    0
FX                    0
N                     0
P                     1
Q                     0
RG                    0
SQ                    0
T                     0
T10                   0
TD                    0
U                     0
VV                    0
WW                    0
dwelling              0
gasPower              6
dtype: int64

In [14]:
df_2 = df.dropna()
df_2.head()

,eMeter,eMeterReturn,eMeterLow,eMeterLowReturn,ePower,ePowerReturn,gasMeter,DD,DR,FF,...,RG,SQ,T,T10,TD,U,VV,WW,dwelling,gasPower
datetime,,,,,,,,,,,,,,,,,,,,,
2017-03-08,2469.029,0.0,2371.596,0.0,125.085200,0.0,2086.006,203.76666,0.0,2.501667,...,0.0,0.0,4.666666,3.050000,2.833333,87.333336,18766.666,1.166667,P01S01W5855,0.000000
2017-03-09,2475.444,0.0,2374.938,0.0,126.119446,0.0,2089.907,217.00000,0.0,2.163333,...,0.0,0.0,3.533333,-1.283333,3.383333,98.500000,9753.333,6.000000,P01S01W5855,0.000000
2017-03-10,2483.384,0.0,2378.290,0.0,140.569440,0.0,2094.775,340.65000,0.0,2.930000,...,0.0,0.0,4.566667,2.516667,3.383333,91.666664,13700.000,1.000000,P01S01W5855,0.000000
2017-03-11,2483.384,0.0,2382.287,0.0,131.451430,0.0,2099.785,144.91666,0.0,3.343333,...,0.0,0.0,7.716667,6.200000,1.766667,65.833336,27433.334,1.166667,P01S01W5855,0.155029
2017-03-12,2483.384,0.0,2391.395,0.0,132.594450,0.0,2102.839,88.95000,0.0,1.840000,...,0.0,0.0,5.733333,1.616667,2.500000,79.333336,18683.334,2.333333,P01S01W5855,0.150146


In [15]:
df_2['gasPower']

datetime
2017-03-08    0.000000
2017-03-09    0.000000
2017-03-10    0.000000
2017-03-11    0.155029
2017-03-12    0.150146
2017-03-13    0.023926
2017-03-14    0.000000
2017-03-15    0.165039
2017-03-16    0.017822
2017-03-17    0.175781
2017-03-18    0.000000
2017-03-19    0.281006
2017-03-20    0.152100
2017-03-21    0.024170
2017-03-22    0.000000
2017-03-23    0.000000
2017-03-24    0.000000
2017-03-25    0.000000
2017-03-26    0.025146
2017-03-27    0.000000
2017-03-28    0.000000
2017-03-29    0.000000
2017-03-30    0.093018
2017-03-31    0.005127
2017-04-01    0.000000
2017-04-02    0.046875
2017-04-03    0.000000
2017-04-04    0.000000
2017-04-08    0.190186
2017-04-09    0.000000
                ...   
2017-04-30    0.226807
2017-05-01    0.094971
2017-05-02    0.035156
2017-05-03    0.000000
2017-05-04    0.000000
2017-05-05    0.000000
2017-05-06    0.521973
2017-05-07    0.235840
2017-05-08    0.002930
2017-05-09    0.000000
2017-05-10    0.006104
2017-05-11    0.000000
20